## Dynamic Programming - Iterative Policy Evaluation

Dynamic programming is a method of computing an optimal policy given a *perfect* model of an environment.

This is an implementation of Example 4.1 in [Policy Evaluation](http://incompleteideas.net/book/ebook/node41.html). The goal is to get to the corners in the fewest steps.

![gridworld](http://incompleteideas.net/book/ebook/imgtmp4.png)

I made a value function that uses iterative policy evaluation to update state values.
The policy used is a greedy policy based on the value function at that moment in time.


In this basic gridworld example, the greedy policy is the optimal policy, so we end up finding the optimal value function $V^*$ through this approach.

In [1]:
from gridworld.world import World, Coordinate, actions, Direction

In [624]:
x_len = 6
y_len = 6
def reward_function(coordinate: Coordinate):
    return -1

def initialize_world(initial_position=Coordinate(2, 3)):
    world = World(x_len, y_len, reward_function, initial_position=initial_position)
    world.print()
    return world

world = initialize_world()

......
......
......
..X...
......
......


Here is the iterative policy evaluation loop.

I use a form of the Bellman equation for the greedy policy $\pi$. 

$$ V_{k+1} = \sum_a \pi(s, a) \sum_{s'} P^a_{ss'} [ R^a_{ss'} + \gamma V_k (s')] $$

- $\pi(s, a)$ is the probabillity of taking an action $a$ from that state $s$. The greedy policy will return a list of actions that are greedy given the value function $V_k$. Since the probability of selecting either of these actions is equal, this manifests in the algorithm by dividing the sum by the count of the possible actions.
- $P^a_{ss'}$ is the probability that taking action $a$ will result in state $s$ transitioning to state $s'$. Our gridworld is deterministic, so this is 1 and is ignored in the algorithm.
- $R_{ss'}^a$ is the reward of the state transition from state $s$ to state $s'$. In this environment, it's always -1.
- $V_k(s')$ is the old value function's determination of the next state value. It's multiplied by $\gamma$ (some number less than 1 and greater than 0) to assure convergence.

I iterate until $\Delta$ is smaller than some small number; it ends up taking only 6 iterations to converge entirely.

In [748]:
import numpy as np
from typing import List

state_value_map = [[0 for x in range(x_len)] for y in range(y_len)]
def value_function(state: Coordinate) -> float:
    return state_value_map[state.x][state.y]

def policy_function(state: Coordinate) -> List[Direction]:
    # Returns a list of actions that maximize the value function. 
    # If there are multiple actions that maximize the value function, return them all.
    max_value = -float('inf')
    max_actions = []
    for action in actions:
        next_coordinate = world.move_direction(state, action)
        value = value_function(next_coordinate)
        if value > max_value:
            max_value = value
            max_actions = [action]
        elif value == max_value:
            max_actions.append(action)
    return max_actions

gamma = 0.9
def update_value_function():
    iterations = 0
    while True:
        iterations += 1
        value_function_delta = 0
        for x in range(x_len):
            for y in range(y_len):
                state = Coordinate(x, y)
                # Assuming a Greedy policy with respect to the current value function (`policy_function`)
                # Incrementally updating the state value map for faster convergence
                old_state = state_value_map[x][y]
                # Get possible actions for the current state
                # If the current state is in the goal state, then the value function is 0
                if state == Coordinate(x_len - 1, y_len - 1):
                    state_value_map[x][y] = 0
                elif state == Coordinate(0, 0):
                    state_value_map[x][y] = 0
                else:
                    possible_actions = policy_function(state)
                    action_count = len(possible_actions)
                    state_value_map[x][y] = float(np.array([-1 + (gamma * value_function(world.move_direction(state, action))) for action in possible_actions]).sum()) / action_count
                value_function_delta = max(value_function_delta, abs(old_state - state_value_map[x][y]))

        if value_function_delta < 0.00001:
            break

    return iterations


In [749]:
def print_state_value_map():
    for row in state_value_map:
        for cell in row:
            print(f"{cell:.1f}", end=", ")
        print()


print("Initial value function:")
print_state_value_map()

iterations = update_value_function()

print(f"After {iterations} iteration of updating the value function:")
print_state_value_map()

Initial value function:
0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
After 6 iteration of updating the value function:
0.0, -1.0, -1.9, -2.7, -3.4, -4.1, 
-1.0, -1.9, -2.7, -3.4, -4.1, -3.4, 
-1.9, -2.7, -3.4, -4.1, -3.4, -2.7, 
-2.7, -3.4, -4.1, -3.4, -2.7, -1.9, 
-3.4, -4.1, -3.4, -2.7, -1.9, -1.0, 
-4.1, -3.4, -2.7, -1.9, -1.0, 0.0, 


In [744]:
world = initialize_world()

def step_with_policy_function():
    directions = policy_function(world.current_position)
    direction = np.random.choice(directions)
    world.move(direction)

steps = 0
while not (world.current_position == Coordinate(x_len - 1, y_len - 1) or world.current_position == Coordinate(0, 0)):
    steps += 1
    step_with_policy_function()

print("-" * 8)
world.print()
print(f"Made it in {steps} steps!")

......
......
......
..X...
......
......
--------
......
......
......
......
......
.....X
Made it in 5 steps!
